In [10]:
import pickle
import random
import time
import os

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from matplotlib import pyplot as plt
import soundfile as sf
import librosa
import librosa.display

tqdm.pandas()

import paule
import paule.paule
from paule import util
from paule.util import (speak, normalize_cp, inv_normalize_cp, normalize_mel_librosa, inv_normalize_mel_librosa, stereo_to_mono, librosa_melspec, pad_same_to_even_seq_length, RMSELoss, mel_to_sig, pad_batch_online)
from paule.models import *
from matplotlib import cm
from matplotlib.lines import Line2D

DIR = os.getcwd()
DEVICE = torch.device('cpu')

In [4]:
DATA_DIR = "/home/tino/ml_cloud_nextcloud/Common_Voice_data/"
RESULT_DICT = "results/"
SAVE_DICT = "saves/"

# Initialize Paule

## add continue data

In [5]:
valid = pd.read_pickle(os.path.join(DATA_DIR, "common_voice_geco_words_test_subset_slim_prot4.pkl"))

In [6]:
random.seed(23082022)
valid_sample = random.sample(range(len(valid)), 12)

In [7]:
continue_data = valid.iloc[valid_sample][["vector", "cp_norm", "melspec_norm_synthesized"]].copy()

KeyError: "['melspec_norm_synthesized'] not in index"

In [ ]:
continue_data["segment_data"] = True

In [ ]:
continue_data

In [12]:
paule_model = paule.paule.Paule(use_somatosensory_feedback=False, continue_data=None, device=DEVICE)

In [13]:
vectors = pd.read_pickle(DATA_DIR + "lexical_embedding_vectors.pkl")

# Prepare Test Data

In [14]:
test_data = pd.read_pickle(DATA_DIR + "common_voice_geco_words_test_subset_slim_prot4.pkl")

## Create Flac Files 

In [15]:
for i,row in test_data.iterrows():
    sr = 48000
    sig = row.wav_rec
    file_name = row.file_name
    label = row.label
    
    path = RESULT_DICT + label + "/"
    if os.path.isdir(path):
        file = path +  file_name + "_" + label + ".flac"
        if os.path.isfile(file):
            j = 1
            while os.path.isfile(file): 
                file = path +  file_name + "_" + label + "%d.flac" % j
                j+=1
        sf.write(file ,sig, sr)
    else:
        os.mkdir(path)
        sf.write(path +  file_name + "_" + label + ".flac" ,sig, sr)

## Random Shuffle Files

In [16]:
files = []
for dic in os.listdir(RESULT_DICT):
    if (".DS_Store" in dic) or ("results" in dic) : # .DS_Store stores custom attributes of its containing folder in macOS
        continue
    else:
        if not os.path.isdir(os.path.join(SAVE_DICT, dic)):
            os.mkdir(os.path.join(SAVE_DICT, dic))
        path = os.path.join(RESULT_DICT, dic)
        for file in os.listdir(path):
            if (".DS_Store" in file) or ("._" in file) or ".flac" not in file:
                continue
            else:
                file = os.path.join(path,file)
                files.append(file)
random.seed(30112021)
random.shuffle(files)

### Omit already planned files

In [17]:
already_planned_files = []
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if "planned" in file and ".flac" in file and not "best" in file: 
                    planned_file = "_".join(file.split("_")[:-1]) + ".flac" 
                    already_planned_files.append(planned_file)

unplanned_files = [file for file in files if file.split("/")[-1] not in already_planned_files]

### Load predictive model already used and further learned during planning 

In [18]:
# load pred_model and pred_optimizer
pred_model = torch.load("", map_location=DEVICE)
optimizer = torch.load("", map_location=DEVICE)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
# load inv_model and inv_optimizer
inv_model = torch.load("", map_location=DEVICE)
inv_optimizer = torch.load("", map_location=DEVICE)

In [ ]:
# load tube_model and tube_optimizer
tube_model = torch.load("", map_location=DEVICE)
tube_optimizer = torch.load("", map_location=DEVICE)

In [ ]:
paule_model.pred_model = pred_model
paule_model.pred_optimizer = optimizer

In [ ]:
paule_model.inv_model = inv_model
paule_model.inv_optimizer = inv_optimizer

In [ ]:
paule_model.tube_model = tube_model
paule_model.tube_optimizer = tube_optimizer

# Start Planning and store results 

In [ ]:
fontsize_title = 35
fontsize_x = 30
fontsize_y = 30
fontsize_params = 20

In [ ]:
continue_learning_inv = True
for i, file in enumerate(unplanned_files):
    i += len(already_planned_files)
    target_acoustic = file #path + "/" + file
    label = file.split("/")[-1].split("_")[-1][:-5]
    target_semvec = vectors[vectors.label == label].vector.iloc[0]
    
    save_file = f"{SAVE_DICT}/{os.path.dirname(file).split('/')[-1]}/{os.path.basename(file)[:-5]}"
    
    results = paule_model.plan_resynth(learning_rate_planning=0.01,
            learning_rate_learning=0.001,
            learning_rate_learning_inv=0.001,
            target_acoustic=target_acoustic,
            target_semvec=target_semvec,
            initialize_from="acoustic",
            objective="acoustic",
            n_outer=10, n_inner=25,
            continue_learning=True,
            continue_learning_inv=continue_learning_inv,
            add_training_data_pred=False,
            add_training_data_inv=True,
            log_ii=1,
            log_semantics=True,
            n_batches=3, batch_size=8, n_epochs=10,
            log_gradients=False,
            plot=save_file, 
            seed=None,
            verbose=True)
    
    # save model and optimizer
    torch.save(paule_model.pred_model, f"{save_file}_{str(i)}_pred_model.pt")
    torch.save(paule_model.pred_optimizer, f"{save_file}_{str(i)}_pred_optimizer.pt")
    
    if continue_learning_inv:
        torch.save(paule_model.inv_model, f"{save_file}_{str(i)}_inv_model.pt")
        torch.save(paule_model.inv_optimizer, f"{save_file}_{str(i)}_inv_optimizer.pt")
    

    # save results without model and optimizer
    with open(f"{save_file}.pkl", 'wb') as pfile:
        pickle.dump(results, pfile)
        
    # save continue data if used
    if not paule_model.continue_data is None:
        with open(f"{SAVE_DICT}/continue_data.pkl", 'wb') as pfile:
            pickle.dump(paule_model.continue_data, pfile)
    
    
    # save best synthesis acoustic
    with open(f"{save_file}_best_synthesis_acoustic.pkl", 'wb') as pfile:
        pickle.dump(paule_model.best_synthesis_acoustic, pfile)
    pfile.close()
    
    prod_sig = paule_model.best_synthesis_acoustic.prod_sig
    sf.write(save_file + "_planned_best_acoustic.flac", prod_sig, 44100)
    del prod_sig
    
    if paule_model.best_synthesis_semantic:
        # save results without model and optimizer
        with open(f"{save_file}_best_synthesis_semantic.pkl", 'wb') as pfile:
            pickle.dump(paule_model.best_synthesis_semantic, pfile)
        pfile.close()
        
        prod_sig = paule_model.best_synthesis_semantic.prod_sig
        sf.write(save_file + "_planned_best_semantic.flac", prod_sig, 44100)
        del prod_sig
    
    if paule_model.use_somatosensory_feedback:
        with open(f"{save_file}_best_synthesis_somatosensory.pkl", 'wb') as pfile:
            pickle.dump(paule_model.best_synthesis_somatosensory, pfile)
        pfile.close()
        
        prod_sig = paule_model.best_synthesis_somatosensory.prod_sig
        sf.write(save_file + "_planned_best_somatosensory.flac", prod_sig, 44100)
        del prod_sig

    plt.cla()
    plt.clf()
    plt.close()

    # save initial and planned flac
    prod_sr = results.prod_sr
    sig_initial = results.initial_sig
    sf.write(save_file + "_initial.flac", sig_initial, prod_sr)
    prod_sig = results.prod_sig
    sf.write(save_file + "_planned.flac", prod_sig, prod_sr)

    # save svgs
    planned_cp = results.planned_cp
    path = save_file + '_svgs/'
    if not os.path.exists(path):
        os.mkdir(path)
    util.export_svgs(util.inv_normalize_cp(planned_cp), path=path)
    
    # save svgs initial 
    initial_cp = results.initial_cp
    path = save_file + '_svgs_initial/'
    if not os.path.exists(path):
        os.mkdir(path)
    util.export_svgs(util.inv_normalize_cp(initial_cp), path=path)

    # ffmpeg -r 80 -width 600 -i tract%05d.svg -i planned_0.flac planned_0.mp4
    # /usr/bin/ffmpeg -r 80 -width 600 -i /home/tino/Documents/phd/projects/paule/results/000003-Wissenschaft_svgs/tract%05d.svg -i results/000003-Wissenschaft_planned.flac planned.mp4


    # save loss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.planned_loss_steps, label="planned loss", c="C0")
    ax.legend()
    fig.savefig(f"{save_file}_loss.png")
    plt.cla()
    plt.clf()
    plt.close()

    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.prod_loss_steps, label="produced mel loss", c="C1")
    ax.plot(results.planned_mel_loss_steps, label="planned mel loss", c="C0")
    ax.legend()
    fig.savefig(f"{save_file}_loss_mel.png")
    plt.cla()
    plt.clf()
    plt.close()

    # save subloss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.vel_loss_steps, label="vel loss", c="C2")
    ax.plot(results.jerk_loss_steps, label="jerk loss", c="C3")
    ax.legend()
    fig.savefig(f"{save_file}_loss_subloss.png")
    plt.cla()
    plt.clf()
    plt.close()

    # save semvec loss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.pred_semvec_loss_steps, label="planned semvec loss", c="C0")
    ax.plot(results.prod_semvec_loss_steps, label="produced semvec loss", c="C1")
    ax.legend()
    fig.savefig(f"{save_file}_loss_semvec.png")
    plt.cla()
    plt.clf()
    plt.close()
    
    if paule_model.use_somatosensory_feedback:
        # save tube loss plot
        fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
        ax.plot(results.prod_tube_loss_steps, label="produced tube loss", c="C0")
        ax.legend()
        fig.savefig(f"{save_file}_loss_tube.png")
        plt.cla()
        plt.clf()
        plt.close()
        
        fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
        ax.plot(results.prod_tube_mel_loss_steps, label="produced tube mel loss", c="C1")
        ax.plot(results.pred_tube_mel_loss_steps, label="planned tube mel loss", c="C0")
        ax.legend()
        fig.savefig(f"{save_file}_loss_tube_mel.png")
        plt.cla()
        plt.clf()
        plt.close()
        
        fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
        ax.plot(results.pred_tube_semvec_loss_steps, label="planned tube semvec loss", c="C0")
        ax.plot(results.prod_tube_semvec_loss_steps, label="produced tube semvec loss", c="C1")
        ax.legend()
        fig.savefig(f"{save_file}_loss_tube_semvec.png")
        plt.cla()
        plt.clf()
        plt.close()
    
    
    # save cps
    # Vocal Tract CPs   
    fig, ax = plt.subplots(facecolor="white", figsize=(15, 10))
    colors = ["C%d" % i for i in range(19)]

    for i in range(4):
        ax.plot(results.planned_cp[:, i], color=colors[i], ls = 'solid', lw = 2)
        ax.plot(results.initial_cp[:, i], color=colors[i], ls = 'dotted', lw = 4)

    legend_elements = [Line2D([0], [0], color='black', ls="solid", lw=2, label='Planned Cp'),#
                       Line2D([0], [0], color='black',ls ="dotted", lw = 4 , label='Inital CP')]

    ax.set_ylim((-1.1, 1.1))
    plt.legend(handles=legend_elements, fontsize=fontsize_params, bbox_to_anchor=(1.0, 1),frameon = False) 
    ax.tick_params(axis='both', labelsize=fontsize_params)
    ax.set_ylabel('Normalized Position' , fontsize=fontsize_y, labelpad=20)
    ax.set_xlabel('Timestep (2.5ms)' , fontsize=fontsize_x, labelpad=20)  
    plt.title("Vocal Tract Cps: '%s'" % save_file.split("/")[-1], fontsize=18, pad=10)
    fig.savefig(f"{save_file}_vocal_tract_cps.png")
    plt.cla()
    plt.clf()
    plt.close()
    
    # Glottis CPs
    fig, ax = plt.subplots(facecolor="white", figsize=(15, 10))
    colors = ["C%d" % i for i in range(19)]

    for i in range(4):
        ax.plot(results.planned_cp[:, 19+i], color=colors[i], ls = 'solid', lw = 2)
        ax.plot(results.initial_cp[:, 19+i], color=colors[i], ls = 'dotted', lw = 4)

    legend_elements = [Line2D([0], [0], color='black', ls="solid", lw=2, label='Planned Cp'),#
                       Line2D([0], [0], color='black',ls ="dotted", lw = 4 , label='Inital CP')]

    ax.set_ylim((-1.1, 1.1))
    plt.legend(handles=legend_elements, fontsize=fontsize_params, bbox_to_anchor=(1.0, 1),frameon = False) 
    ax.tick_params(axis='both', labelsize=fontsize_params)
    ax.set_ylabel('Normalized Position' , fontsize=fontsize_y, labelpad=20)
    ax.set_xlabel('Timestep (2.5ms)' , fontsize=fontsize_x, labelpad=20)
    plt.title("Glottis Cps: '%s'" % save_file.split("/")[-1], fontsize=18, pad=10)
    fig.savefig(f"{save_file}_glottis_cps.png")
    plt.cla()
    plt.clf()
    plt.close()

## Store final losses to txt 

In [ ]:
file_names = []
with open(SAVE_DICT + '/results_loss.txt', 'w') as txt:
    first_row = True
    for dic in os.listdir(SAVE_DICT):
        if ".DS_Store" in dic:
            continue
        else:
            path = os.path.join(SAVE_DICT,dic)
            if os.path.isdir(path):
                for file in os.listdir(path):
                    if ".pkl" in file and "best_synthesis" not in file: 
                        with open(os.path.join(path,file), 'rb') as f:
                            results = pickle.load(f)
                            file_name = "_".join(file.split("_")[:-1])
                            loss = results.planned_loss_steps[-1]
                            prod_mel_loss = results.prod_loss_steps[-1]
                            pred_mel_loss = results.planned_mel_loss_steps[-1]
                            vel_loss = results.vel_loss_steps[-1]
                            jerk_loss = results.jerk_loss_steps[-1]
                            
                            prod_semvec_loss = results.prod_semvec_loss_steps[-1]
                            pred_semvec_loss = results.pred_semvec_loss_steps[-1] 
                            if hasattr(results, "prod_tube_loss_steps"):
                                prod_tube_loss = results.prod_tube_loss_steps[-1]
                                prod_tube_mel_loss = results.prod_tube_mel_loss_steps[-1]
                                pred_tube_mel_loss = results.pred_tube_mel_loss_steps[-1]
                                prod_tube_semvec_loss = results.prod_tube_semvec_loss_steps[-1]
                                pred_tube_semvec_loss = results.pred_tube_semvec_loss_steps[-1]
                        file_names.append(file_name)    
                        if first_row:
                            header = "filename prod_mel_loss pred_mel_loss vel_loss jerk_loss prod_semvec_loss pred_semvec_loss"
                            if hasattr(results, 'prod_tube_loss_steps'):
                                txt.write(header + " prod_tube_loss prod_tube_mel_loss pred_tube_mel_loss prod_tube_semvec_loss pred_tube_semvec_loss\n") 
                            else:
                                txt.write(header +"\n")
                            first_row = False
                        if hasattr(results, 'prod_tube_loss_steps'):
                            txt.write(file_name + " %.5f %.5f %.5f %.5f %.5f %.5f %.5f %.5f %.5f %.5f %.5f\n" % (prod_mel_loss, pred_mel_loss, vel_loss, jerk_loss, prod_semvec_loss, pred_semvec_loss, prod_tube_loss, prod_tube_mel_loss, pred_tube_mel_loss, prod_tube_semvec_loss, pred_tube_semvec_loss))
                        else:    
                            txt.write(file_name + " %.5f %.5f %.5f %.5f %.5f %.5f\n" % (prod_mel_loss, pred_mel_loss, vel_loss, jerk_loss, prod_semvec_loss, pred_semvec_loss))

# Plot Model Loss for continued learning 

In [ ]:
pred_model_loss = []
inv_model_loss = []
tube_model_loss = []

for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file and "best_synthesis" not in file: 
                    with open(os.path.join(path,file), 'rb') as f:
                        results = pickle.load(f)
                        pred_model_loss.append(results.pred_model_loss)
                        if hasattr(results, "inv_model_loss"):
                            inv_model_loss.append(results.inv_model_loss)
                        if hasattr(results, "tube_model_loss"):
                            tube_model_loss.append(results.tube_model_loss)

In [ ]:
fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
ax.plot(np.array(pred_model_loss).flatten(), label = "pred_model loss", c = "C0")
#ax.plot(range(len(model_loss)),np.repeat(np.mean(model_loss),len(model_loss)) ,c = "C1")
ax.legend()

# Data Wrangling with Final Results

In [ ]:
vectors = pd.read_pickle(DATA_DIR + "lexical_embedding_vectors.pkl")

In [ ]:
ix = 0
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file and "best_synthesis" not in file: 
                    label = dic
                    target_semvec = vectors[vectors.label == label].iloc[0].vector
                    with open(os.path.join(path,file), 'rb') as f:
                        results = pickle.load(f)
                        file_name = "_".join(file.split("_")[:-1])
                        planned_cp = results.planned_cp
                        initial_cp = results.initial_cp
                        initial_sig = results.initial_sig
                        initial_prod_mel = results.initial_prod_mel
                        initial_pred_mel = results.initial_pred_mel
                        target_sig = results.target_sig
                        target_sr = results.target_sr
                        target_mel = results.target_mel
                        prod_sig = results.prod_sig
                        prod_sr = results.prod_sr 
                        prod_mel = results.prod_mel
                        pred_mel = results.pred_mel
                        initial_prod_semvec = results.initial_prod_semvec
                        initial_pred_semvec = results.initial_pred_semvec
                        prod_semvec = results.prod_semvec
                        pred_semvec = results.pred_semvec                      
                        prod_mel_loss = results.prod_loss_steps
                        planned_loss = results.planned_loss_steps
                        planned_mel_loss = results.planned_mel_loss_steps
                        vel_loss = results.vel_loss_steps
                        jerk_loss = results.jerk_loss_steps
                        prod_semvec_loss = results.prod_semvec_loss_steps
                        pred_semvec_loss = results.pred_semvec_loss_steps
                        prod_semvec_steps = results.prod_semvec_steps 
                        pred_semvec_steps = results.pred_semvec_steps
                        pred_model_loss = results.pred_model_loss

                        if hasattr(results, "inv_model_loss"):
                            inv_model_loss = results.inv_model_loss

                        if hasattr(results, "tube_model_loss"):
                            initial_prod_tube = results.initial_prod_tube 
                            initial_pred_tube = results.initial_pred_tube 
                            initial_prod_tube_mel = results.initial_prod_tube_mel
                            initial_pred_tube_mel = results.initial_pred_tube_mel
                            prod_tube = results.prod_tube
                            pred_tube = results.pred_tube
                            prod_tube_mel = results.prod_tube_mel
                            pred_tube_mel = results.pred_tube_mel
                            initial_prod_tube_semvec = results.initial_prod_tube_semvec
                            initial_pred_tube_semvec = results.initial_pred_tube_semvec
                            prod_tube_semvec = results.prod_tube_semvec
                            pred_tube_semvec = results.pred_tube_semvec
                            prod_tube_loss = results.prod_tube_loss_steps
                            pred_tube_mel_loss = results.pred_tube_mel_loss_steps
                            prod_tube_mel_loss = results.prod_tube_mel_loss_steps
                            pred_tube_semvec_loss = results.pred_tube_semvec_loss_steps
                            prod_tube_semvec_loss = results.prod_tube_semvec_loss_steps
                            prod_tube_semvec_steps = results.prod_tube_semvec_steps
                            pred_tube_semvec_steps = results.pred_tube_semvec_steps
                            tube_model_loss = results.tube_model_loss

                        f.close()

                        if ix == 0:
                            columns = ['file_name','label', 'planned_cp', 'initial_cp', 
                                                  'initial_sig','initial_prod_mel', 'initial_pred_mel', 
                                                  'target_sig', 'target_sr','target_mel',
                                                  'prod_sig','prod_sr','prod_mel', 'pred_mel', 
                                                  'initial_prod_semvec', 'initial_pred_semvec',
                                                  'target_semvec',
                                                  'prod_semvec', 'pred_semvec',
                                                  'prod_mel_loss', 'planned_loss', 'planned_mel_loss', 'vel_loss', 'jerk_loss',
                                                  'prod_semvec_loss', 'pred_semvec_loss',
                                                  'prod_semvec_steps', 'pred_semvec_steps',
                                                  'pred_model_loss']
                            
                            if hasattr(results, "inv_model_loss"):
                                columns += ["inv_model_loss"]

                            if hasattr(results, "tube_model_loss"):
                                columns += ["initial_prod_tube", "initial_pred_tube", "initial_prod_tube_mel","initial_pred_tube_mel",
                                            "prod_tube", "pred_tube", "prod_tube_mel", "pred_tube_mel", 
                                             "initial_prod_tube_semvec", "initial_pred_tube_semvec", 
                                            "prod_tube_semvec", "pred_tube_semvec", "prod_tube_loss", 
                                            "pred_tube_mel_loss", "prod_tube_mel_loss", "pred_tube_semvec_loss",
                                            "prod_tube_semvec_loss", "prod_tube_semvec_steps", "pred_tube_semvec_steps", "tube_model_loss"]
 
                            
                            final_results = pd.DataFrame(columns=columns)

                    
                        data = [file_name, label, planned_cp, initial_cp , 
                                initial_sig, initial_prod_mel, initial_pred_mel, 
                                target_sig, target_sr, target_mel,
                                prod_sig, prod_sr, prod_mel, pred_mel, 
                                initial_prod_semvec, initial_pred_semvec,
                                target_semvec, prod_semvec, pred_semvec,
                                prod_mel_loss, planned_loss, planned_mel_loss, 
                                vel_loss, jerk_loss, prod_semvec_loss, pred_semvec_loss,
                                prod_semvec_steps, pred_semvec_steps, pred_model_loss]
                            
                        if hasattr(results, "inv_model_loss"):
                            data += [inv_model_loss]

                        if hasattr(results, "tube_model_loss"):
                            data += [initial_prod_tube, initial_pred_tube, initial_prod_tube_mel,initial_pred_tube_mel,
                                     prod_tube, pred_tube, prod_tube_mel, pred_tube_mel, 
                                     initial_prod_tube_semvec, initial_pred_tube_semvec, 
                                     prod_tube_semvec, pred_tube_semvec, prod_tube_loss, 
                                     pred_tube_mel_loss, prod_tube_mel_loss, pred_tube_semvec_loss,
                                     prod_tube_semvec_loss, prod_tube_semvec_steps, pred_tube_semvec_steps, tube_model_loss]
            
                        final_results.loc[ix] = data
                        ix+=1

In [ ]:
final_results.to_pickle(SAVE_DICT+"/final_results.pkl", protocol=4)

# Remove all individual results

In [ ]:
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file: 
                    os.remove(os.path.join(path,file))
                if ".pt" in file:
                    if not int(file.split("_")[-3]) == (len(final_results)-1):
                        os.remove(os.path.join(path,file))